In [4]:
from dashboard.logic.io import *
from dashboard.logic.plots import components, donut_chart
from dashboard.logic.constants import getMetaDataDict

# 3rd part imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from bokeh.io import output_notebook, show

output_notebook()

Loading BokehJS ...

In [1]:
a = 'tere'

# CACHING

### Download Workbook as .xlsx with timestamp

In [5]:
from dashboard.logic.constants import nav_names
from dashboard.logic.io import *

#dct = nav_names['2023 Forecasts & Risks']
dct = nav_names['Investment Allocation Examples']


path = "/home/tonu/Documents/apps/dashboard_invest/dashboard/cache/example_portf"
df = read_gsheet(, header=None)
mask_df = df.apply(lambda x: x.str.startswith("IDEAL PORTFOLIO"))


# Plotting

In [10]:
import pandas as pd
import numpy as np

# bokeh
from bokeh.io import show, output_notebook
from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend, LabelSet, LegendItem, HoverTool, AnnularWedge
from bokeh.layouts import column

In [61]:
from dashboard.logic.plots import components

def donut_chart(
    df: pd.DataFrame, 
    x: str, 
    y: str,
    x_hover: str=None,
    y_hover: str=None, 
    inner_radius: float=0.4,
    outer_radius: float=0.8,
    label_distance: float=3,
    x_range: tuple[float, float]=(-1, 1.0),
    percentage_decimal: int=1,
    fig_height: int=350,
    background_color: str='#212529',
    pallette: dict=Category10,
    sizing_mode='scale_width',
    hover_tooltip: str='default',
    legend_place: str='center',
    fig_kwargs: dict={},
    wedge_kwargs: dict=dict(line_width=3, alpha=0.7),
    legend_kwargs: dict=dict(location='center', click_policy="hide",
                             label_text_color='white', border_line_width=0,
                             inactive_fill_color='#9fcf2e', inactive_fill_alpha=0.15,
                             background_fill_alpha=0),
    label_kwargs: dict=dict(text_font_size='10pt', text_align='center')
    ):
    
    
    # sort df by "y"
    df = df.sort_values(by=y, ignore_index=True)
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(percentage_decimal)
    df['percentage_hover'] = df['percentage_number'].astype(str)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x:.{percentage_decimal}f}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * label_distance * outer_radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * label_distance * outer_radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, 
               toolbar_location=None, 
               x_range=x_range,
               y_range=(-1.0, 1.0),
               sizing_mode=sizing_mode,
               **fig_kwargs)
    
    legend_items = []
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs renderers
        wedge = p.annular_wedge(x=0, y=0, inner_radius=inner_radius, outer_radius=outer_radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, **wedge_kwargs,
                        fill_color=color, hover_fill_color=color,
                        line_color=background_color, hover_line_color=background_color,
                        line_alpha=1, hover_alpha=1, hover_line_alpha=1)
        
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         source=source, level='glyph', text_color=background_color, **label_kwargs)
        
        x_hover = x if x_hover is None else x_hover
        y_hover = y if y_hover is None else y_hover
        
        hover_tooltip = hover_tooltip if hover_tooltip != 'default' else \
            f"""
                <div>
                    <p style="margin:0;font-weight:bold;color:grey;">@{x_hover}</p>
                    <p style="padding:0;margin:0;font-weight:bold;">$@{y_hover}{{0,0.00}} (@percentage_hover%)</p>
                </div>
                <style>
                    .bk-tooltip {{
                        background-color: red!important;
                    }}
                </style>
            """
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=hover_tooltip))

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))

    # legend
    legend = Legend(items=legend_items,  **legend_kwargs)
    
    p.add_layout(legend, place=legend_place)

    
    # figure attributes
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    
    p.min_border=0
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color = p.background_fill_color = p.border_fill_color = background_color

    return show(p)

h_tooltip = f"""
                <div>
                    <p style="margin:0;font-weight:bold;color:grey;">@Sector</p>
                    <p style="padding:0;margin:0;font-weight:bold;text-align:center;">@percentage_hover{{0}}%</p>
                </div>
                <style>
                    .bk-root .bk-tooltip {{
                        background-color: red;
                    }}
                </style>
            """

donut_chart(
        df=df_sectors_plot.iloc[1:,],
        x='Sector',
        y='Proportion',
        sizing_mode='fixed',
        background_color='#2C2B2B',
        percentage_decimal=0,
        fig_height=500,
        label_distance=3.1,
        label_kwargs=dict(text_font_size='12pt', text_align='center', text_font_style='bold'),
        legend_place='center',
        fig_kwargs={'width':500}
    )
#obj_js, obj_div = components(obj)